# Visulalizing and Calculating Parameters on a Skew-T

A powerful tool is being developed that will have the same functionality as GEMPAK, but be done in the Python programming langauage. One example of this is the ability to plot Skew-Ts using the Python module [MetPy](https://unidata.github.io/MetPy/latest/index.html). However, there is greatly functionality than that contained in GEMPAK, as you can use MetPy to calculate a whole host of variables from the data plotted on the Skew-T (e.g., CAPE, CIN, LCL, LFC, EL). This notebook is a brief introduction to how that can be done.

To plot a skew-T, you need to specify the date and time (in UTC) and choose an upper-air station location. The locations of current sounding sites can be found at http://weather.rap.ucar.edu/upper

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.plots import SkewT
from metpy.units import units, pandas_dataframe_to_unit_arrays
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

In [ ]:
# Set date that you want
# Data goes back to the 1970's
date = datetime(2007, 5, 4, 18)

# Set station ID, there are different stations back in the day
# Current station IDs found at http://weather.rap.ucar.edu/upper
station = 'DDC'

# Use Siphon module to grab data from remote server
df = WyomingUpperAir.request_data(date, station)

In [ ]:
# Create dictionary of unit arrays
data = pandas_dataframe_to_unit_arrays(df)

# Isolate united arrays from dictionary to individual variables
p = data['pressure']
T = data['temperature']
Td = data['dewpoint']
u = data['u_wind']
v = data['v_wind']

In [ ]:
# Plot a skew-T image

fig = plt.figure(figsize=(10, 10))

skew = SkewT(fig)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Plot wind barbs, skipping every other one
skew.plot_barbs(p[::2], u[::2], v[::2], y_clip_radius=0.03)

# Set sensible axis limits
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 50)

# Add the relevant special lines
skew.plot_dry_adiabats(t0=np.arange(233,555,10)*units.K)
skew.plot_moist_adiabats(color='tab:green')
skew.plot_mixing_lines(color='tab:blue')

# Plot some titles
plt.title('Station: K{}'.format(station), loc='left')
plt.title('Skew-T/Log-p', loc='center')
plt.title('{} UTC'.format(date), loc='right')

# Show the plot
# plt.savefig('skewt_image.png', dpi=150)
plt.show()

## Calculations of Sounding Parameters

Once we have our data, we can calculate any of the skew-T variables we desire, such as LCL using the functions available in MetPy (https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html#soundings).

In [ ]:
# Calculate LCL pressure and temperature
# Note: since lcl function returns two values, we can have
# two variables to the left of the equal sign

lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
print('LCL: {:.2f}'.format(lcl_pressure))

In [ ]:
# Calculate full surface parcel profile and add to plot as black line
prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

# Calculate the CAPE and CIN of the profile
cape, cin = mpcalc.cape_cin(p, T, Td, prof)

print('CAPE: {:.2f}'.format(cape))
print('CIN: {:.2f}'.format(cin))

In [ ]:
fig = plt.figure(figsize=(10, 10))

skew = SkewT(fig)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Plot barbs skipping to every other barb
skew.plot_barbs(p[::2], u[::2], v[::2], y_clip_radius=0.03)

# Set sensible axis limits
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 50)

# Plot the skew-T parcel temperature profile
skew.plot(p, prof, 'k', linewidth=2)

# Plot a line marker at the LCL level
skew.plot(lcl_pressure, lcl_temperature, 'k_', markersize=24, 
          markeredgewidth=3, markerfacecolor='black')

# Add the relevant special lines
skew.plot_dry_adiabats(t0=np.arange(233,555,10)*units.K)
skew.plot_moist_adiabats(color='tab:green')
skew.plot_mixing_lines(color='tab:blue')

# Plot some titles
plt.title('Station: K{}'.format(station), loc='left')
plt.title('Skew-T/Log-p', loc='center')
plt.title('{} UTC'.format(date), loc='right')

# Show the plot
plt.show()



## Reading Local Data

Our radiosonde launch system provides data in a number of formats. At the top of this notebook there is a function that is set up to ready the SHARPPY ready file format that is available on our sounding page at http://bergeron.valpo.edu/soundings/. To use this data, select the appropriate sounding from the drop down menus near the top of the page, then click on the "SharpPy Ready File", this will open a new window, copy and past the full web address into the read function below.

In [ ]:
def read_SHARPPY(filename):
    import pandas as pd

    df = pd.read_csv(filename,
                     skiprows=6, names=['pressure','height','temperature','dewpoint','direction','speed'], skipfooter=1,
                     engine='python')
    df['u_wind'], df['v_wind'] = mpcalc.wind_components(df['speed'].values * units.knots,
                                                        df['direction'].values * units.degrees)

    df.units = dict(pressure='hPa', height='meter', temperature='degC', dewpoint='degC',
                    direction='degrees', speed='knot', u_wind='knot', v_wind='knot')
    return df

In [ ]:
# Set station and date/time for plotting purposes
station = 'VUM'
date = datetime(2019, 3, 5, 10, 57)

# Get back similar dataframe to what you get from the Wyoming archive
df2 = read_SHARPPY('http://bergeron.valpo.edu/soundings/launches/20190305_01/20190305_01_SHARPPY.txt')

# Get united arrays from our dataframe
data2 = pandas_dataframe_to_unit_arrays(df2)

# Isolate united arrays for calculation and plotting
p = data2['pressure']
T = data2['temperature']
Td = data2['dewpoint']
u = data2['u_wind']
v = data2['v_wind']

Make the skew-T Plot

In [ ]:
fig = plt.figure(figsize=(10, 10))

skew = SkewT(fig)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Create a slice variable to help reduce the total
# number of wind barbs
wind_slice = slice(None, None, 15)
skew.plot_barbs(p[wind_slice], u[wind_slice], v[wind_slice], y_clip_radius=0.03)

# Set sensible axis limits
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 50)

# Add the relevant special lines
skew.plot_dry_adiabats(t0=np.arange(233,555,10)*units.K)
skew.plot_moist_adiabats(color='tab:green')
skew.plot_mixing_lines(color='tab:blue')

# Plot some titles
plt.title('Station: K{}'.format(station), loc='left')
plt.title('Skew-T/Log-p', loc='center')
plt.title('{} UTC'.format(date), loc='right')

# Show the plot
plt.show()